# Bouw een regressiemodel: data voorbereiden en visualiseren

## **Lineaire regressie voor pompoenen - Les 2**
#### Introductie

Nu je beschikt over de tools die je nodig hebt om te beginnen met het bouwen van machine learning-modellen met Tidymodels en de Tidyverse, ben je klaar om vragen te stellen over je data. Het is erg belangrijk om te begrijpen hoe je de juiste vragen stelt om de mogelijkheden van je dataset optimaal te benutten wanneer je met data werkt en ML-oplossingen toepast.

In deze les leer je:

-   Hoe je je data voorbereidt voor het bouwen van modellen.

-   Hoe je `ggplot2` gebruikt voor datavisualisatie.

De vraag die je beantwoord wilt hebben, bepaalt welk type ML-algoritmes je zult gebruiken. En de kwaliteit van het antwoord dat je terugkrijgt, hangt sterk af van de aard van je data.

Laten we dit zien door een praktische oefening te doen.


<p >
   <img src="../../images/unruly_data.jpg"
   width="700"/>
   <figcaption>Kunstwerk door @allison_horst</figcaption>


<!--![Kunstwerk door \@allison_horst](../../../../../../2-Regression/2-Data/images/unruly_data.jpg)<br>Kunstwerk door \@allison_horst-->


## 1. Pompoengegevens importeren en het Tidyverse oproepen

We hebben de volgende pakketten nodig om deze les te analyseren en verwerken:

-   `tidyverse`: Het [tidyverse](https://www.tidyverse.org/) is een [verzameling van R-pakketten](https://www.tidyverse.org/packages) die ontworpen zijn om datawetenschap sneller, eenvoudiger en leuker te maken!

Je kunt ze installeren met:

`install.packages(c("tidyverse"))`

Het onderstaande script controleert of je de benodigde pakketten hebt om deze module te voltooien en installeert ze voor je als er enkele ontbreken.


In [ ]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))
pacman::p_load(tidyverse)

Laten we enkele pakketten opstarten en de [gegevens](https://github.com/microsoft/ML-For-Beginners/blob/main/2-Regression/data/US-pumpkins.csv) laden die voor deze les zijn verstrekt!


In [ ]:
# Load the core Tidyverse packages
library(tidyverse)

# Import the pumpkins data
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv")


# Get a glimpse and dimensions of the data
glimpse(pumpkins)


# Print the first 50 rows of the data set
pumpkins %>% 
  slice_head(n =50)

Een snelle `glimpse()` laat meteen zien dat er lege waarden zijn en een mix van strings (`chr`) en numerieke data (`dbl`). De `Date` is van het type karakter en er is ook een vreemde kolom genaamd `Package` waar de data een mix is van `sacks`, `bins` en andere waarden. De data is, eerlijk gezegd, een beetje een rommeltje 😤.

Het is eigenlijk niet heel gebruikelijk om een dataset te krijgen die volledig klaar is om direct een ML-model mee te maken. Maar geen zorgen, in deze les leer je hoe je een ruwe dataset kunt voorbereiden met behulp van standaard R-bibliotheken 🧑‍🔧. Je leert ook verschillende technieken om de data te visualiseren. 📈📊
<br>

> Een opfrisser: De pipe-operator (`%>%`) voert bewerkingen uit in logische volgorde door een object door te geven aan een functie of expressie. Je kunt de pipe-operator zien als "en dan" in je code.


## 2. Controleer op ontbrekende gegevens

Een van de meest voorkomende problemen waarmee datawetenschappers te maken hebben, is incomplete of ontbrekende gegevens. R vertegenwoordigt ontbrekende of onbekende waarden met een speciale sentinelwaarde: `NA` (Not Available).

Hoe kunnen we weten of het gegevensframe ontbrekende waarden bevat?
<br>
-   Een eenvoudige manier is om de basis R-functie `anyNA` te gebruiken, die de logische objecten `TRUE` of `FALSE` retourneert.


In [ ]:
pumpkins %>% 
  anyNA()

Geweldig, het lijkt erop dat er wat gegevens ontbreken! Dat is een goed startpunt.

-   Een andere manier is om de functie `is.na()` te gebruiken, die aangeeft welke individuele kolomelementen ontbreken met een logische `TRUE`.


In [ ]:
pumpkins %>% 
  is.na() %>% 
  head(n = 7)

Oké, het is gelukt, maar met een grote dataset zoals deze zou het inefficiënt en praktisch onmogelijk zijn om alle rijen en kolommen afzonderlijk te controleren😴.

-   Een meer intuïtieve manier zou zijn om de som van de ontbrekende waarden per kolom te berekenen:


In [ ]:
pumpkins %>% 
  is.na() %>% 
  colSums()

Veel beter! Er ontbreken wat gegevens, maar misschien maakt dat niet uit voor de taak die voor ons ligt. Laten we eens kijken wat verder onderzoek oplevert.

> Naast de geweldige sets pakketten en functies, heeft R een zeer goede documentatie. Gebruik bijvoorbeeld `help(colSums)` of `?colSums` om meer te weten te komen over de functie.


## 3. Dplyr: Een Grammatica voor Datamanipulatie

<p >
   <img src="../../images/dplyr_wrangling.png"
   width="569"/>
   <figcaption>Illustratie door @allison_horst</figcaption>


<!--![Illustratie door \@allison_horst](../../../../../../2-Regression/2-Data/images/dplyr_wrangling.png)<br/>Illustratie door \@allison_horst-->


[`dplyr`](https://dplyr.tidyverse.org/), een pakket in de Tidyverse, is een grammatica voor datamanipulatie die een consistente set werkwoorden biedt om je te helpen de meest voorkomende uitdagingen bij datamanipulatie op te lossen. In deze sectie gaan we enkele werkwoorden van dplyr verkennen!
<br>


#### dplyr::select()

`select()` is een functie in het pakket `dplyr` waarmee je kolommen kunt kiezen om te behouden of uit te sluiten.

Om je data frame eenvoudiger te maken om mee te werken, kun je meerdere kolommen verwijderen met behulp van `select()`, waarbij je alleen de kolommen behoudt die je nodig hebt.

Bijvoorbeeld, in deze oefening zullen we ons richten op de kolommen `Package`, `Low Price`, `High Price` en `Date`. Laten we deze kolommen selecteren.


In [ ]:
# Select desired columns
pumpkins <- pumpkins %>% 
  select(Package, `Low Price`, `High Price`, Date)


# Print data set
pumpkins %>% 
  slice_head(n = 5)

#### dplyr::mutate()

`mutate()` is een functie in het pakket `dplyr` waarmee je kolommen kunt aanmaken of aanpassen, terwijl de bestaande kolommen behouden blijven.

De algemene structuur van mutate is:

`data %>%   mutate(new_column_name = what_it_contains)`

Laten we `mutate` in de praktijk gebruiken met de `Date`-kolom door de volgende bewerkingen uit te voeren:

1.  Converteer de datums (momenteel van het type karakter) naar een maandformaat (dit zijn Amerikaanse datums, dus het formaat is `MM/DD/YYYY`).

2.  Haal de maand uit de datums en sla deze op in een nieuwe kolom.

In R maakt het pakket [lubridate](https://lubridate.tidyverse.org/) het eenvoudiger om met datum-tijdgegevens te werken. Dus laten we `dplyr::mutate()`, `lubridate::mdy()`, `lubridate::month()` gebruiken om bovenstaande doelen te bereiken. We kunnen de Date-kolom verwijderen, aangezien we deze niet meer nodig hebben in de volgende bewerkingen.


In [ ]:
# Load lubridate
library(lubridate)

pumpkins <- pumpkins %>% 
  # Convert the Date column to a date object
  mutate(Date = mdy(Date)) %>% 
  # Extract month from Date
  mutate(Month = month(Date)) %>% 
  # Drop Date column
  select(-Date)

# View the first few rows
pumpkins %>% 
  slice_head(n = 7)

Woohoo! 🤩

Laten we een nieuwe kolom `Price` maken, die de gemiddelde prijs van een pompoen weergeeft. Nu gaan we het gemiddelde nemen van de kolommen `Low Price` en `High Price` om de nieuwe kolom Price in te vullen.
<br>


In [ ]:
# Create a new column Price
pumpkins <- pumpkins %>% 
  mutate(Price = (`Low Price` + `High Price`)/2)

# View the first few rows of the data
pumpkins %>% 
  slice_head(n = 5)

Yeees!💪

"Maar wacht!", zul je zeggen na het snel doorlopen van de hele dataset met `View(pumpkins)`, "Er is hier iets vreemds!"🤔

Als je naar de kolom `Package` kijkt, worden pompoenen in veel verschillende configuraties verkocht. Sommige worden verkocht in `1 1/9 bushel` maten, andere in `1/2 bushel` maten, sommige per pompoen, sommige per pond, en sommige in grote dozen met verschillende breedtes.

Laten we dit verifiëren:


In [ ]:
# Verify the distinct observations in Package column
pumpkins %>% 
  distinct(Package)

Geweldig!👏

Pompoenen lijken erg moeilijk consistent te wegen, dus laten we ze filteren door alleen pompoenen te selecteren met de string *bushel* in de kolom `Package` en dit in een nieuw gegevensframe `new_pumpkins` te plaatsen.
<br>


#### dplyr::filter() en stringr::str_detect()

[`dplyr::filter()`](https://dplyr.tidyverse.org/reference/filter.html): maakt een subset van de data die alleen **rijen** bevat die voldoen aan jouw voorwaarden, in dit geval pompoenen met de string *bushel* in de `Package`-kolom.

[stringr::str_detect()](https://stringr.tidyverse.org/reference/str_detect.html): detecteert de aanwezigheid of afwezigheid van een patroon in een string.

De [`stringr`](https://github.com/tidyverse/stringr)-package biedt eenvoudige functies voor veelvoorkomende bewerkingen met strings.


In [ ]:
# Retain only pumpkins with "bushel"
new_pumpkins <- pumpkins %>% 
       filter(str_detect(Package, "bushel"))

# Get the dimensions of the new data
dim(new_pumpkins)

# View a few rows of the new data
new_pumpkins %>% 
  slice_head(n = 5)

Je kunt zien dat we het hebben teruggebracht tot ongeveer 415 rijen met gegevens over pompoenen per schepel.🤩
<br>


#### dplyr::case_when()

**Maar wacht! Er is nog iets te doen**

Heb je gemerkt dat de hoeveelheid per bushel per rij verschilt? Je moet de prijzen normaliseren zodat je de prijs per bushel laat zien, en niet per 1 1/9 of 1/2 bushel. Tijd om wat rekenwerk te doen om dit te standaardiseren.

We gebruiken de functie [`case_when()`](https://dplyr.tidyverse.org/reference/case_when.html) om de kolom Price te *muteren* op basis van bepaalde voorwaarden. `case_when` stelt je in staat om meerdere `if_else()`-verklaringen te vectoriseren.


In [ ]:
# Convert the price if the Package contains fractional bushel values
new_pumpkins <- new_pumpkins %>% 
  mutate(Price = case_when(
    str_detect(Package, "1 1/9") ~ Price/(1 + 1/9),
    str_detect(Package, "1/2") ~ Price/(1/2),
    TRUE ~ Price))

# View the first few rows of the data
new_pumpkins %>% 
  slice_head(n = 30)

Nu kunnen we de prijs per eenheid analyseren op basis van hun bushel-meting. Al deze studie over bushels van pompoenen laat echter zien hoe `belangrijk` het is om `de aard van je gegevens te begrijpen`!

> ✅ Volgens [The Spruce Eats](https://www.thespruceeats.com/how-much-is-a-bushel-1389308) hangt het gewicht van een bushel af van het type product, omdat het een volumemeting is. "Een bushel tomaten, bijvoorbeeld, zou 56 pond moeten wegen... Bladeren en groenten nemen meer ruimte in met minder gewicht, dus een bushel spinazie weegt slechts 20 pond." Het is allemaal behoorlijk ingewikkeld! Laten we ons niet druk maken over het omrekenen van bushels naar ponden, en in plaats daarvan de prijs per bushel bepalen. Al deze studie over bushels van pompoenen laat echter zien hoe belangrijk het is om de aard van je gegevens te begrijpen!
>
> ✅ Is het je opgevallen dat pompoenen die per halve bushel worden verkocht erg duur zijn? Kun je achterhalen waarom? Hint: kleine pompoenen zijn veel duurder dan grote, waarschijnlijk omdat er veel meer van zijn per bushel, gezien de ongebruikte ruimte die wordt ingenomen door één grote holle taartpompoen.


Belangrijkste regels:  
1. Verplaats de kolom `Month` naar de eerste positie, vóór de kolom `Package`.  
2. Gebruik de functie `dplyr::relocate()` om de kolomvolgorde te wijzigen.  


In [ ]:
# Create a new data frame new_pumpkins
new_pumpkins <- new_pumpkins %>% 
  relocate(Month, .before = Package)

new_pumpkins %>% 
  slice_head(n = 7)

Goed gedaan!👌 Je hebt nu een schoon en overzichtelijk dataset waarop je je nieuwe regressiemodel kunt bouwen!
<br>


## 4. Gegevensvisualisatie met ggplot2

<p >
   <img src="../../images/data-visualization.png"
   width="600"/>
   <figcaption>Infographic door Dasani Madipalli</figcaption>


<!--![Infographic door Dasani Madipalli](../../../../../../2-Regression/2-Data/images/data-visualization.png){width="600"}-->

Er is een *wijs* gezegde dat als volgt gaat:

> "De eenvoudige grafiek heeft meer informatie gebracht naar de geest van de data-analist dan enig ander hulpmiddel." --- John Tukey

Een deel van de rol van een datawetenschapper is om de kwaliteit en aard van de gegevens waarmee ze werken te demonstreren. Om dit te doen, maken ze vaak interessante visualisaties, zoals diagrammen, grafieken en tabellen, die verschillende aspecten van de gegevens tonen. Op deze manier kunnen ze visueel relaties en hiaten laten zien die anders moeilijk te ontdekken zijn.

Visualisaties kunnen ook helpen bij het bepalen van de meest geschikte machine learning-techniek voor de gegevens. Een spreidingsdiagram dat een lijn lijkt te volgen, geeft bijvoorbeeld aan dat de gegevens een goede kandidaat zijn voor een lineaire regressie-oefening.

R biedt verschillende systemen voor het maken van grafieken, maar [`ggplot2`](https://ggplot2.tidyverse.org/index.html) is een van de meest elegante en veelzijdige. Met `ggplot2` kun je grafieken samenstellen door **onafhankelijke componenten te combineren**.

Laten we beginnen met een eenvoudig spreidingsdiagram voor de kolommen Prijs en Maand.

In dit geval beginnen we met [`ggplot()`](https://ggplot2.tidyverse.org/reference/ggplot.html), leveren een dataset en esthetische mapping (met [`aes()`](https://ggplot2.tidyverse.org/reference/aes.html)) en voegen vervolgens lagen toe (zoals [`geom_point()`](https://ggplot2.tidyverse.org/reference/geom_point.html)) voor spreidingsdiagrammen.


In [ ]:
# Set a theme for the plots
theme_set(theme_light())

# Create a scatter plot
p <- ggplot(data = new_pumpkins, aes(x = Price, y = Month))
p + geom_point()

Is dit een nuttige grafiek 🤷? Verrast er iets je eraan?

Het is niet bijzonder nuttig, aangezien het alleen je gegevens weergeeft als een spreiding van punten in een bepaalde maand.
<br>


### **Hoe maken we het nuttig?**

Om grafieken nuttige gegevens te laten weergeven, moet je de gegevens meestal op een bepaalde manier groeperen. Bijvoorbeeld, in ons geval zou het vinden van de gemiddelde prijs van pompoenen per maand meer inzicht geven in de onderliggende patronen in onze gegevens. Dit brengt ons bij nog een **dplyr**-overzicht:

#### `dplyr::group_by() %>% summarize()`

Groepsaggregatie in R kan eenvoudig worden berekend met

`dplyr::group_by() %>% summarize()`

-   `dplyr::group_by()` verandert de analyseeenheid van de volledige dataset naar individuele groepen, zoals per maand.

-   `dplyr::summarize()` maakt een nieuw data frame met één kolom voor elke groeperingsvariabele en één kolom voor elke samenvattende statistiek die je hebt gespecificeerd.

Bijvoorbeeld, we kunnen `dplyr::group_by() %>% summarize()` gebruiken om de pompoenen te groeperen op basis van de **Maand**-kolommen en vervolgens de **gemiddelde prijs** voor elke maand te berekenen.


In [ ]:
# Find the average price of pumpkins per month
new_pumpkins %>%
  group_by(Month) %>% 
  summarise(mean_price = mean(Price))

Bondig!✨

Categorische kenmerken zoals maanden worden beter weergegeven met een staafdiagram 📊. De lagen die verantwoordelijk zijn voor staafdiagrammen zijn `geom_bar()` en `geom_col()`. Raadpleeg `?geom_bar` voor meer informatie.

Laten we er een maken!


In [ ]:
# Find the average price of pumpkins per month then plot a bar chart
new_pumpkins %>%
  group_by(Month) %>% 
  summarise(mean_price = mean(Price)) %>% 
  ggplot(aes(x = Month, y = mean_price)) +
  geom_col(fill = "midnightblue", alpha = 0.7) +
  ylab("Pumpkin Price")

🤩🤩Dit is een nuttigere datavisualisatie! Het lijkt erop dat de hoogste prijs voor pompoenen in september en oktober voorkomt. Komt dat overeen met jouw verwachting? Waarom wel of niet?

Gefeliciteerd met het afronden van de tweede les 👏! Je hebt je gegevens voorbereid voor het bouwen van een model en vervolgens meer inzichten ontdekt met behulp van visualisaties!



---

**Disclaimer**:  
Dit document is vertaald met behulp van de AI-vertalingsservice [Co-op Translator](https://github.com/Azure/co-op-translator). Hoewel we streven naar nauwkeurigheid, willen we u erop wijzen dat geautomatiseerde vertalingen fouten of onnauwkeurigheden kunnen bevatten. Het originele document in de oorspronkelijke taal moet worden beschouwd als de gezaghebbende bron. Voor kritieke informatie wordt professionele menselijke vertaling aanbevolen. Wij zijn niet aansprakelijk voor misverstanden of verkeerde interpretaties die voortvloeien uit het gebruik van deze vertaling.
